## 201810794 이은후

In [1]:
%%writefile src/Bank.sol
pragma solidity 0.5.1;
contract BankTransfer {
    uint balance;
    uint increment;
    address payable owner;
    constructor() public {
        balance = address(this).balance;
    }
    function setOwner(address payable _owner) public {
        owner = _owner;
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public {
        increment = balance;
        owner.transfer(balance);
    }
    function getBalance() public view returns(uint, uint) {
        return(address(this).balance, balance);
    }
    function forwardTo(address payable _receiver) public payable {
        uint amount = 333;
        increment = amount;
        balance -= amount;
        _receiver.transfer(333);
    }
    function incrementAmount() public view returns(uint) {
        return increment;
    }
}

Overwriting src/Bank.sol


### 컴파일

In [2]:
!solc --abi --bin src/Bank.sol


======= src/Bank.sol:BankTransfer =======
Binary: 
608060405234801561001057600080fd5b503073ffffffffffffffffffffffffffffffffffffffff163160008190555061034e8061003e6000396000f3fe608060405260043610610072576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461007757806313af4035146100a957806327d8ad88146100fa578063853828b61461013e578063a4a049dd14610155578063b6b55f2514610180575b600080fd5b34801561008357600080fd5b5061008c6101ae565b604051808381526020018281526020019250505060405180910390f35b3480156100b557600080fd5b506100f8600480360360208110156100cc57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506101d4565b005b61013c6004803603602081101561011057600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610218565b005b34801561014a57600080fd5b50610153610282565b005b34801561016157600080fd5b5061016a6102f8565b6040518082815260200191505060405180910390f35b6101ac6004803603602081101561019657600080fd5b8101908080

### 컨트랙 배포

In [3]:
%%writefile src/BankDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="608060405234801561001057600080fd5b503073ffffffffffffffffffffffffffffffffffffffff163160008190555061034e8061003e6000396000f3fe608060405260043610610072576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461007757806313af4035146100a957806327d8ad88146100fa578063853828b61461013e578063a4a049dd14610155578063b6b55f2514610180575b600080fd5b34801561008357600080fd5b5061008c6101ae565b604051808381526020018281526020019250505060405180910390f35b3480156100b557600080fd5b506100f8600480360360208110156100cc57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506101d4565b005b61013c6004803603602081101561011057600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610218565b005b34801561014a57600080fd5b50610153610282565b005b34801561016157600080fd5b5061016a6102f8565b6040518082815260200191505060405180910390f35b6101ac6004803603602081101561019657600080fd5b8101908080359060200190929190505050610302565b005b6000803073ffffffffffffffffffffffffffffffffffffffff1631600054915091509091565b80600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b600061014d9050806001819055508060008082825403925050819055508173ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f1935050505015801561027d573d6000803e3d6000fd5b505050565b600054600181905550600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc6000549081150290604051600060405180830381858888f193505050501580156102f5573d6000803e3d6000fd5b50565b6000600154905090565b803414151561031057600080fd5b8060008082825401925050819055505056fea165627a7a72305820f48b75e872290d78a83ecf58c4a25cd89c4f7701f1407ca94921924edb9acb0a0029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x52cadd19f40f5f5f544f33fde2f8fc118862e4f9", gas: 364124, gasPrice: '100000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/BankDeploy.js


In [4]:
!node src/BankDeploy.js

0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4


### 사용

문제1: 입금 11111wei, 222wei

In [5]:
%%writefile src/BankUse1_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.deposit(11111).send({from:"0x52cadd19f40f5f5f544f33fde2f8fc118862e4f9",gas:800000,value:11111});
myBank.methods.deposit(222).send({from:"0x52cadd19f40f5f5f544f33fde2f8fc118862e4f9",gas:800000,value:222});

Overwriting src/BankUse1_1.js


In [6]:
!node src/BankUse1_1.js

잔고 11333wei 출력

In [7]:
%%writefile src/BankUse1_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.getBalance().call().then(function(value){console.log(value);});

Overwriting src/BankUse1_2.js


In [8]:
!node src/BankUse1_2.js

Result { '0': '11333', '1': '11333' }


문제2: 자신의 2번째 계정으로 계좌이체 333wei

In [9]:
%%writefile src/BankUse2_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.forwardTo('0x81f6c6b03cbab015ea7142a0ac6bccf00aac2407').send({from:"0x52cadd19f40f5f5f544f33fde2f8fc118862e4f9",gas:800000,value:333});

Overwriting src/BankUse2_1.js


In [10]:
!node src/BankUse2_1.js

컨트랙 잔고 11000wei, 잔고 증가분(333wei) 출력

In [11]:
%%writefile src/BankUse2_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.getBalance().call().then(function(value){console.log(value);});
myBank.methods.incrementAmount().call().then(function(value){console.log(value);});

Overwriting src/BankUse2_2.js


In [12]:
!node src/BankUse2_2.js

333
Result { '0': '11333', '1': '11000' }


문제3: 전액 인출(11000wei)

In [13]:
%%writefile src/BankUse3_1.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.withdrawAll().send({from:"0x52cadd19f40f5f5f544f33fde2f8fc118862e4f9",gas:800000});

Overwriting src/BankUse3_1.js


In [14]:
!node src/BankUse3_1.js

자신의 잔고 증가분(11000wei) 출력

In [15]:
%%writefile src/BankUse3_2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_owner","type":"address"}],"name":"setOwner","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"incrementAmount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var myBank = new web3.eth.Contract(_abiArray, '0xcf2d3b59837242e67D2F2D4B8B7C16f6429C9cB4');
myBank.methods.getBalance().call().then(function(value){console.log(value);});
myBank.methods.incrementAmount().call().then(function(value){console.log(value);});

Overwriting src/BankUse3_2.js


In [16]:
!node src/BankUse3_2.js

Result { '0': '333', '1': '11000' }
11000
